In [ ]:
import json
import pandas as pd

# فایل ورودی را از آدرس مشخصی میخوانیم
with open('service1/input.json') as file:
    data = json.load(file)

#فایل خوانده شده را از حالت جیسون خارج می‌کنیم و فرمت ورودی تاریخ را درست میکنیم
def read_json_time_series(dict_data):
    j_data = json.dumps(dict_data)
    data = pd.read_json(j_data)
    data.time = pd.to_datetime(data.time, unit='ms')
    return data
#داده‌ی خوانده شده چند بخش دارد. این بخش‌ها را از هم جدا می‌کنیم.
df = read_json_time_series(data['data'])
config = data['config']

#یک تابه تعریف میکنیم تا براساس ماهیت ورودی بتوانیم درون‌یابی را انجام دهیم
def linear_interpolation(data, config):
    if config['time'] == 'daily':
        data = data.set_index('time')
        data = data.resample('D')
        data = data.interpolate(method=config['interpolation'])
        data.reset_index(inplace=True)

    elif config['time'] == 'monthly':
        data = data.set_index('time')
        data = data.resample('M')
        data = data.interpolate(method=config['interpolation'])
        data.reset_index(inplace=True)

    else:
        data = None

    return data

# با استفاده از تابعی که بالا تعریف کردیم با دادن ورودی و کانفیگ، داده‌های میس شده را درون یابی می‌کنیم و سپس به جیسون تبدیل می‌کنیم
result = linear_interpolation(df, config)
result = result.to_json()

#داده‌ی جیسون شده را در همان آدرس به صورت فایل ذخیره می‌کنیم
with open('service1/output.json', "w") as outfile:
    outfile.write(result)